In [1]:
import pandas as pd
import numpy as np
from glob import glob
import random
import os
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf

In [8]:
energy = pd.read_csv("../data/solar/energy.csv")
ulsan_fcst = pd.read_csv("../data/solar/ulsan_fcst_data.csv")

In [9]:
def convert_time(x):
    Ymd, HMS = x.split(' ')
    H, M, S = HMS.split(':')
    H = str(int(H)-1)
    HMS = ':'.join([H, M, S])
    return ' '.join([Ymd, HMS])

energy['time'] = energy['time'].apply(lambda x:convert_time(x))

In [10]:
energy

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2018-03-01 0:00:00,0.0,0.0,0,0
1,2018-03-01 1:00:00,0.0,0.0,0,0
2,2018-03-01 2:00:00,0.0,0.0,0,0
3,2018-03-01 3:00:00,0.0,0.0,0,0
4,2018-03-01 4:00:00,0.0,0.0,0,0
...,...,...,...,...,...
25627,2021-01-31 19:00:00,0.0,0.0,0,0
25628,2021-01-31 20:00:00,0.0,0.0,0,0
25629,2021-01-31 21:00:00,0.0,0.0,0,0
25630,2021-01-31 22:00:00,0.0,0.0,0,0


In [11]:
def to_date(x):
    return pd.DateOffset(hours=x)


ulsan_fcst['Forecast_time'] = pd.to_datetime(ulsan_fcst['Forecast time'])
ulsan_fcst_14 = ulsan_fcst[ulsan_fcst['Forecast_time'].dt.hour==14]
ulsan_fcst_14 = ulsan_fcst_14[(ulsan_fcst_14['forecast']>=10)&(ulsan_fcst_14['forecast']<=33)]
ulsan_fcst_14['Forecast_time'] = ulsan_fcst_14['Forecast_time'] + ulsan_fcst_14['forecast'].map(to_date)
ulsan_fcst_14 = ulsan_fcst_14[['Forecast_time', 'Temperature', 'Humidity', 'WindSpeed', 'WindDirection', 'Cloud']]


/tmp/ipykernel_26888/3285780281.py:8: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  ulsan_fcst_14['Forecast_time'] = ulsan_fcst_14['Forecast_time'] + ulsan_fcst_14['forecast'].map(to_date)


In [20]:
ulsan_fcst_14["Forecast_time"] = pd.to_datetime(ulsan_fcst_14["Forecast_time"])

In [46]:
ulsan_fcst_14_ = pd.DataFrame()
ulsan_fcst_14_['Forecast_time'] = pd.date_range(start='2018-03-02 00:00:00', end='2021-03-01 23:00:00', freq='h')

In [23]:
ulsan_fcst_14.shape, ulsan_fcst_14_.shape

((8768, 6), (26304, 1))

In [47]:
ulsan_fcst_14_ = pd.merge(ulsan_fcst_14_, ulsan_fcst_14, on='Forecast_time', how='outer')

In [48]:
ulsan_fcst = ulsan_fcst_14_.interpolate()

In [27]:
ulsan_fcst['Forecast_time'] = ulsan_fcst['Forecast_time'].astype(str)

In [32]:
ulsan_fcst

,Forecast_time,Temperature,Humidity,WindSpeed,WindDirection,Cloud
0,2018-03-02 00:00:00,0.000000,40.000000,1.500000,318.000000,2.000000
1,2018-03-02 01:00:00,-0.333333,41.666667,1.600000,314.666667,2.000000
2,2018-03-02 02:00:00,-0.666667,43.333333,1.700000,311.333333,2.000000
3,2018-03-02 03:00:00,-1.000000,45.000000,1.800000,308.000000,2.000000
4,2018-03-02 04:00:00,-1.333333,45.000000,1.766667,303.333333,1.666667
...,...,...,...,...,...,...
26299,2021-03-01 19:00:00,15.333333,80.000000,5.100000,31.000000,4.000000
26300,2021-03-01 20:00:00,14.666667,80.000000,5.900000,23.000000,4.000000
26301,2021-03-01 21:00:00,14.000000,80.000000,6.700000,15.000000,4.000000
26302,2021-03-01 22:00:00,14.000000,80.000000,6.700000,15.000000,4.000000


In [42]:
ulsan_energy = pd.DataFrame()
ulsan_energy['Forecast_time'] = pd.date_range(start='2018-03-01 00:00:00', end='2021-01-31 23:00:00', freq='h')

In [40]:
ulsan_energy.shape, energy.shape

((25632,), (25632, 5))

In [43]:
ulsan_energy["energy"] = energy["ulsan"]

In [49]:
data = pd.merge(ulsan_fcst, ulsan_energy, on="Forecast_time")

In [56]:
train, test = data.iloc[:-744], data.iloc[-744:]

In [57]:
train.to_csv("../data/solar/train.csv", index=False)
test.to_csv("../data/solar/test.csv", index=False)